In [1]:
from gurobipy import *

# The formulations in this module allow the user to provide the auxiliary variables. This give the user the option
#   To use the same auxiliary variables among different terms.

# Model assumptions:
#   ALL VARIABLES MUST HAVE BOTH UPPER AND LOWER BOUNDS, EXPLICITLY SET AS GUROBI ATTRIBUTES.
#   No general constraints.
#   Model has been updated, so that all variables and constraints have been added.
class Model_qcqp:
    NMDT = 0
    DNMDT = 1
    UNIVAR = 2

    def __init__(self, mdl_o, L=3, L1=10, method=DNMDT, ldas=[0.5]):
        self.mdl_o = mdl_o
        self.L = L
        self.L1 = L1  # Applies only to univariate formulation
        self.method = method

        # ldas apply only to D-NMDT
        if len(ldas) == 0:
            ldas = [0.5]
        self.ldas = ldas 

        # Read in model variables and constraints
        self.xs_o = mdl_o.getVars()
        self.lcs_o = mdl_o.getConstrs()
        self.qcs_o = mdl_o.getQConstrs()
        self.I = range(len(self.xs_o))
        self.J = range(1, self.L+1)
        self.J1 = range(1, self.L1+1)

        #self.xtoi = {self.xs_o[i]: i for i in self.I}

        # Create the model
        self.mdl = Model()
        mdl = self.mdl
        # get data (bounds, etc) for original variables, then add to new model. For now, only bnds; names+etc later.

        # Initialize variables
        I = self.I
        xs_o = self.xs_o
        self.xs = mdl.addVars(I)
        xs = self.xs

        self.xotox = {xs_o[i]: xs[i] for i in I}

        # Copy over variables attributes
        for i in I:
            self.copy_var_attrs(xs[i], xs_o[i])

        # Update model so that variables can be used as keys
        mdl.update()

        # Get variable bounds
        gattr = lambda attr, var: mdl_o.getAttr(attr, [var])[0]
        self.xbnds = {xs[i]: (gattr("lb", xs_o[i]), gattr("ub", xs_o[i])) for i in I}

        # dictionaries for z-variables used to model z=x^2 or z=xy.
        self.xy_vdict = {}
        self.xsq_vdict = {}

        # dictionaries for auxiliary variables
        #   Binary variables, shared by all methods
        self.alphas = {}

        # sawtooth-based auxiliary definitions
        #   Stuff for single product terms only
        self.zplus = {}  # auxiliary LP-vars z for z>=(x+y)^2
        self.zplus_g = {}  # auxiliary LP-vars g for z>=(x+y)^2
        self.zminus = {}  # auxiliary LP-vars z for z>=(x-y)^2:
        self.zminus_g = {}  # auxiliary LP-vars g for z>=(x-y)^2
        #   Stuff for lots of terms
        self.gs = {}  # auxiliary vars for sawtooth-based individual quadratic terms z=xy or z<=xy

        # D-NMDT auxiliary definitions
        #   Stuff for single product terms only
        self.vs = {}  # auxiliary vars for alpha_x*(y-stuff)
        self.ws = {}  # auxiliary vars for alpha_y*(z-stuff)
        self.delzs = {}  # auxiliary vars for deltax * deltay stuff
        #   Stuff for lots of terms
        self.delxs = {}  # auxiliary vars for the deltax terms

        # Auxiliary constraints
        self.xy_constrs = {}
        self.xsq_constrs = {}

        # Process stuff to prep model for solving
        self.process_lconstrs()
        self.process_qconstrs()
        self.process_obj()

    def copy_var_attrs(self, x, xo):
        # copy all modifiable variable attributes
        mdl = self.mdl
        mdl_o = self.mdl_o
        gattr = lambda attr: mdl_o.getAttr(attr, [xo])[0]
        sattr = lambda attr, val: mdl.setAttr(attr, [x], [val])

        attr_list = 'lb,ub,vtype,varname,vtag,varhintval,VarHintPri,BranchPriority'.split(',')
        for attr1 in attr_list:
            sattr(attr1, gattr(attr1))
        # omitting simplex-related stuff for now: vbasis, pstart; these may cause errors
        # Omitting 'partition' for now: setting at all seems to spark partition algorithm
        # Omitting 'start' for now

    def process_lconstrs(self):
        mdl = self.mdl
        mdl_o = self.mdl_o
        for lc in self.lcs_o:
            LHS = self.process_lexpr(mdl_o.getRow(lc))
            RHS = lc.qcrhs
            sense = lc.lcsense

            if sense == '=':
                mdl.addConstr(LHS == RHS)
            elif sense == '>':
                mdl.addConstr(LHS >= RHS)
            else:
                mdl.addConstr(LHS <= RHS)

    def process_qconstrs(self):
        mdl = self.mdl
        mdl_o = self.mdl_o
        for qc in self.qcs_o:
            LHS = self.process_qexpr(mdl_o.getQCRow(qc))
            RHS = qc.qcrhs
            sense = qc.qcsense

            if sense == '=':
                mdl.addConstr(LHS == RHS)
            elif sense == '>':
                mdl.addConstr(LHS >= RHS)
            else:
                mdl.addConstr(LHS <= RHS)

    def process_obj(self):
        mdl = self.mdl
        mdl_o = self.mdl_o

        obj = self.process_qexpr(mdl_o.getObjective())
        sense = mdl_o.ModelSense

        mdl.setObjective(obj, sense)

    def process_qexpr(self, qexpr):
        xy_vdict = self.xy_vdict
        xsq_vdict = self.xsq_vdict
        mdl = self.mdl
        xotox = self.xotox

        result_expr = 0

        # process quadratic part
        nqt = qexpr.size()   # number of quadratic terms
        for j in range(nqt):

            coeff = qexpr.getCoeff(j)

            var1 = xotox[qexpr.getVar1(j)]
            var2 = xotox[qexpr.getVar2(j)]

            if var1 is var2:
                # Then this is a pure quadratic term
                if var1 not in xsq_vdict:
                    # TODO: model z = var1*var2 using some method. Add to dict.
                    xsq_vdict[var1] = mdl.addVar(lb=-GRB.INFINITY, name=f'{var1.varName}_sq')
                    self.DNMDTsq(var1)

                result_expr += coeff * xsq_vdict[var1]
            else:
                if (var1, var2) in xy_vdict:
                    result_expr += coeff * xy_vdict[var1, var2]
                elif (var2, var1) in xy_vdict:
                    result_expr += coeff * xy_vdict[var2, var1]
                else:
                    # TODO: model z=var1*var2 using some method (NMDT, D-NMDT, or a univariate reformulation). Then
                    #   add to dict.
                    #     NMDT is harder to implement, as we will need to determine which variables to discretize.
                    #     If using NMDT, we may require the user to specify a list of variables to be discretized, then
                    #       use some (simple/arbitrary) method to choose more to discretize if needed.
                    #     To discretize the fewest variables possible with NMDT for general problems, we need to
                    #       solve the maximal independent set problem, which is NP-hard (oof) to get the largest set of
                    #       variables to not discretize. (Dr. Hildebrand says it solves fast in practice.) Note that we
                    #       must always discretize variables with pure quadratic terms (i.e. diagonal of graph matrix
                    #       may have ones).
                    xy_vdict[var1, var2] = mdl.addVar(lb=-GRB.INFINITY, name=f'{var1.varName}_times_{var2.varName}')
                    self.DNMDT(var1, var2)
                    result_expr += coeff * xy_vdict[var1, var2]

        # process linear expression
        lexpr = qexpr.getLinExpr()
        result_expr += self.process_lexpr(lexpr)

        return result_expr

    def process_lexpr(self, lexpr):
        xotox = self.xotox

        result_expr = 0

        # process linear part
        nl = lexpr.size()
        for j in range(nl):
            coeff = lexpr.getCoeff(j)
            var = xotox[lexpr.getVar(j)]

            result_expr += coeff*var

        # process constant part
        c = lexpr.getConstant()
        result_expr += c

        return result_expr

    # Method definitions

    def McCormick(self, x, y, z, bndx, bndy):
        # McCormick envelopes for z=xy
        # get model, bounds
        mdl = self.mdl
        (lx, ux) = bndx
        (ly, uy) = bndy

        # Add McCormick envelopes
        c1 = mdl.addConstr(z >= x*ly + y*lx - lx*ly)
        c2 = mdl.addConstr(z >= x*uy + y*ux - ux*uy)
        c3 = mdl.addConstr(z <= x*ly + y*ux - ux*ly)
        c4 = mdl.addConstr(z <= x*uy + y*lx - lx*uy)

        return [c1, c2, c3, c4]

    def McCormicksq(self, x, y, bndx):
        # get model, bounds
        mdl = self.mdl
        (lx, ux) = bndx

        # Add McCormick envelopes
        c1 = mdl.addConstr(y >= 2*x*lx - lx**2)
        c2 = mdl.addConstr(y >= 2*x*ux - ux**2)
        c3 = mdl.addConstr(y <= x*(lx+ux) - ux*lx)

        return [c1, c2, c3]

    def DNMDT(self, x, y):
        # Map xh,xy to [0,1], then apply DNMDT to the mapped interval, to model z=xy

        mdl = self.mdl
        L = self.L
        J = self.J
        ldas = self.ldas
        alphas = self.alphas
        delxs = self.delxs
        xy_constrs = self.xy_constrs
        xy_vdict = self.xy_vdict

        xname = x.varname
        yname = y.varname

        # TODO: do this step in update method instead.
        #   for key in xy_constrs: for constr in xy_constrs[key]: mdl.remove(constr)
        if (x, y) in xy_constrs and 0:
            for constr in xy_constrs[x, y]:
                mdl.remove(constr)

        xy_constrs[x, y] = []
        xyc = xy_constrs[x, y]

        for var in (x, y):
            if var not in alphas:
                alphas[var] = mdl.addVars(J, vtype=GRB.BINARY, name=f'{var.varname}_alpha')
                delxs[var] = mdl.addVar(lb=0, ub=2**(-L), name=f'{var.varname}_delx')

        vs = self.vs
        ws = self.ws
        delzs = self.delzs

        delzs[x, y] = mdl.addVar(lb=-GRB.INFINITY, name=f'{xname}_{yname}_delz')

        for lda in ldas:
            # Prepare to model both lda=0 and lda=1
            vs[x, y, lda] = mdl.addVars(J, lb=-GRB.INFINITY, name=f'{xname}_{yname}_v_{lda}')
            ws[x, y, lda] = mdl.addVars(J, lb=-GRB.INFINITY, name=f'{xname}_{yname}_w_{lda}')


        delzh = delzs[x, y]

        alpha = alphas[x]
        beta = alphas[y]

        delxh = delxs[x]
        delyh = delxs[y]

        z = xy_vdict[x, y]

        xlb = x.lb
        ylb = y.lb
        xub = x.ub
        yub = y.ub

        lx = xub-xlb
        ly = yub-ylb

        xh = (x-xlb)/lx
        yh = (y-ylb)/ly
        zh = (z-lx*xh*ylb-ly*yh*xlb-xlb*ylb)/(lx*ly)

        c1 = mdl.addConstr(xh == sum(2**(-j)*alpha[j] for j in J) + delxh)
        c2 = mdl.addConstr(yh == sum(2**(-j)*beta[j] for j in J) + delyh)
        xyc += [c1, c2]

        McCormick = self.McCormick
        for lda in ldas:
            v = vs[x, y, lda]
            w = ws[x, y, lda]
            xyc += [mdl.addConstr(zh == sum(2**(-j)*(v[j]+w[j]) for j in J) + delzh)]

            for j in J:
                # Apply DNMDT with the user-specified value of lambda
                xyc += McCormick(lda*delyh+(1-lda)*yh, alpha[j], v[j], [0, lda*2**(-L) + (1-lda)], [0, 1])
                xyc += McCormick((1-lda)*delxh+lda*xh, beta[j], w[j], [0, (1-lda)*2**(-L) + lda], [0, 1])
        xyc += McCormick(delxh, delyh, delzh, [0, 2**(-L)], [0, 2**(-L)])

    def DNMDTsq(self, x):
        # Apply the y=x^2 version of DNMDT. Note that there is no longer any need for lda.
        mdl = self.mdl
        L = self.L
        J = self.J
        alphas = self.alphas
        delxs = self.delxs
        xsq_constrs = self.xsq_constrs
        xsq_vdict = self.xsq_vdict

        xname = x.varname

        # TODO: do this step in update method instead.
        #   for key in xy_constrs: for constr in xy_constrs[key]: mdl.remove(constr)
        if x in xsq_constrs and 0:
            for constr in xsq_constrs[x]:
                mdl.remove(constr)

        xsq_constrs[x] = []
        xsqc = xsq_constrs[x]

        if x not in alphas:
            alphas[x] = mdl.addVars(J, vtype=GRB.BINARY, name=f'{xname}_alpha')
            delxs[x] = mdl.addVar(lb=0, ub=2**(-L), name=f'{xname}_delx')

        vs = self.vs
        delzs = self.delzs

        vs[x] = mdl.addVars(J, lb=-GRB.INFINITY, name=f'{xname}_v')
        delzs[x] = mdl.addVar(lb=-GRB.INFINITY, name=f'{xname}_delz')

        v = vs[x]
        delz = delzs[x]
        alpha = alphas[x]
        delx = delxs[x]

        z = xsq_vdict[x]

        xlb = x.lb
        xub = x.ub

        lx = xub-xlb

        c1 = mdl.addConstr(x == lx * sum(2**(-j)*alpha[j] for j in J) + delx + xlb)
        c2 = mdl.addConstr(z == lx * sum(2**(-j)*v[j] for j in J) + delz + xlb*(x+delx))

        McCormicksq = self.McCormicksq
        McCormick = self.McCormick
        xsqc += [c1, c2]
        for j in J:
            xsqc += McCormick(delx+x, alpha[j], v[j], [xlb, xub + 2**(-L) * lx], [0, 1])
        xsqc += McCormicksq(delx, delz, [0, 2**(-L) * lx])



In [2]:
from glob import glob as glb
from os import path
from gurobipy import *

fsplit = path.splitext

fpath = r'.\boxQP\LP-data\extended2-LP\*.lp'
fpaths = glb(fpath)
for fpath in fpaths:
    #(fname, _) = fsplit(fpath)
    #print(fname+_)

    m = read(fpath)

    doTest = 0
    if doTest:
        m = Model()
        x = m.addVar(lb=0, ub=1)
        y = m.addVar(lb=0, ub=1)
        z = m.addVar(lb=0, ub=1)

        m.setObjective(x*y+x*z-2*y*z)

        m.setParam(GRB.Param.NonConvex, 2)
        m.setParam(GRB.Param.TimeLimit, 1)
        m.optimize()

    vs = m.getVars()
    for i in range(len(vs)):
        m.setAttr("vtype", [vs[i]], [GRB.CONTINUOUS])

    m.update()

    mdl_holder = Model_qcqp(m, L=1, ldas=[0.5])
    mdl = mdl_holder.mdl

    mdl.setParam(GRB.Param.TimeLimit, 20)
    #mdl.setParam(GRB.Param.OutputFlag, 0)


    mdl.optimize()


    #mdl.computeIIS()
    #mdl.write("modelOops.ilp")

    break


Academic license - for non-commercial use only - expires 2022-08-29
Using license file C:\Users\Ben Beach\gurobi.lic
Read LP format model from file .\boxQP\LP-data\extended2-LP\spar125-025-1.lp
Reading time = 0.00 seconds
: 0 rows, 125 columns, 0 nonzeros
Changed value of parameter TimeLimit to 20.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 29706 rows, 8320 columns and 75185 nonzeros
Model fingerprint: 0xb2b7403e
Variable types: 8195 continuous, 125 integer (125 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [5e-01, 1e+00]
  RHS range        [3e-01, 2e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 11775 rows and 1997 columns
Presolve time: 0.10s
Presolved: 17931 rows, 6323 columns, 49764 nonzeros
Variable types: 6198 continuous, 125 inte